In [ ]:
#!import "Model.ipynb"

In [ ]:
// Create a Jinaga client for the portal
var jinagaClient = JinagaClient.Create(opt =>
{
    opt.HttpEndpoint = new Uri("http://localhost:8000/jinaga");
});

In [ ]:
// // Create an environment and make the logged in user an administrator.
// var (user, profile) = await jinagaClient.Login();

// var environment = await jinagaClient.SingleUse(async principal =>
// {
//     var environment = await jinagaClient.Fact(new Environment(principal, "test"));
//     await jinagaClient.Fact(new Administrator(user, environment, DateTime.UtcNow));
//     return environment;
// });

// System.Text.Json.JsonSerializer.Serialize(environment.creator.publicKey)

In [ ]:
string publicKey = "-----BEGIN PUBLIC KEY-----\r\nMIIBCgKCAQEAo/bSdIPICOFDYuhDLbMYvkxb7wD0CdI8aCH+6AQKL5FkhEsCVoyM\r\nQ2aElw7nbmp9zNfzvHUQ008CDguUcbFM967iMN6Nkm749LHvgZlWExxHaTXQj1En\r\nOabwgACSYp3BmfiQuzK0/j16Z5QqQpsADE3hXuk0fAiDaZShiD5qa7OBva3yqASY\r\nrJ1mlBp4xb47PMJr1ava+1Trdqkx//LXgAUfFpytDkgjbY3YRHyfSlpk0k/BlhiA\r\ndEsjqyyVscrONrqnsWyrY6vz8fuuW5K1l/92U7J9581XEveAHpYpAV+5lwwyGLfc\r\nXSgFHL4QqSlVUcCjiKTKPLSCQ03AGBV8owIDAQAB\r\n-----END PUBLIC KEY-----\r\n";

var environment = new Environment(
    new User(publicKey),
    "test"
);

In [ ]:
// var servicePrincipalPublicKey = "-----BEGIN PUBLIC KEY-----\r\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvvadtWv8WldS\u002B1EPgyoy\r\nJ37mXQEhSmqhqYzfNhBrj5AIjhqGshkU4DvzqHPkvOLDHKfbUVl\u002BlC0Mo3S2aTHq\r\nivZtnSHN3fYW3MA12Ne8\u002BTogpqk1dBHbg\u002BfnQI6bnSmsG\u002Bkw/ZdwtRlC/i55AjXG\r\np/dwlzjwQsvjE8J/1w\u002BQBBAtBCNmcqp0qzH9CFhmyUxOmAVH/NTcfWmn6zRd5ITI\r\n7RseKr2CRl0iM9sNOUT3OR6AEjF4DG4rv9fGHFqW/U6hGKFUg4OwcxbN15lCyVWd\r\nbwtXV1noFkUVGCd9S\u002BmQ0uVqa38lATTOHFvpLshJk7jhgEejqUeL7FdEstGAhiZz\r\n3QIDAQAB\r\n-----END PUBLIC KEY-----\r\n";

// var servicePrincipal = await jinagaClient.Fact(new ServicePrincipal(
//     new User(servicePrincipalPublicKey),
//     environment,
//     DateTime.UtcNow
// ));

// servicePrincipal

In [ ]:
var servicePrincipalsInEnvironment = Given<Environment>.Match((environment, facts) =>
    from servicePrincipal in facts.OfType<ServicePrincipal>()
    where servicePrincipal.environment == environment &&
        !facts.OfType<RevokeServicePrincipal>().Any(revoke => revoke.servicePrincipal == servicePrincipal)
    select servicePrincipal
);

var servicePrincipals = await jinagaClient.Query(servicePrincipalsInEnvironment, environment);

jinagaClient.RenderFacts(servicePrincipals)

In [ ]:
var subscriptionsInEnvironment = Given<Environment>.Match((environment, facts) =>
    from subscription in facts.OfType<Subscription>()
    where subscription.environment == environment
    select new
    {
        subscription,
        userIdentities = facts.OfType<SubscriptionUserIdentity>().Where(sui => sui.subscription == subscription),
        plans = facts.OfType<SubscriptionPlan>().Where(sp => sp.subscription == subscription),
        activations = facts.OfType<Activate>().Where(a => a.subscription == subscription),
        deactivations = facts.OfType<Deactivate>().Where(d => d.activate.subscription == subscription),
        suspensions = facts.OfType<Suspend>().Where(s => s.subscription == subscription),
        reinstatements = facts.OfType<Reinstate>().Where(r => r.suspend.subscription == subscription),
        renewals = facts.OfType<Renew>().Where(r => r.subscription == subscription)
    }
);

var subscriptions = await jinagaClient.Query(subscriptionsInEnvironment, environment);

jinagaClient.RenderFacts(subscriptions)